In [1]:
# connect mysql
import pymysql
conn = pymysql.connect(host = 'localhost', user='ys9925', password='1Q2w3e4r!@', db='crime_prediction', charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

In [2]:
# select from Dataset - Choose scenario

# Season
num = int(input('1.Spring 2.Summer 3.Fall 4.Winter '))
if num == 1: season = 'Spring'
elif num == 2: season = "Summer" 
elif num == 3: season = "Fall" 
else: season = "Winter"

# Time    
num = int(input("1.Morning 2.Afternoon 3.Evening 4.Midnight "))
if num == 1: time = "Morning"  
elif num == 2: time = "Afternoon" 
elif num == 3: time = "Evening" 
else: time = "Midnight"
    
# Weather    
num = int(input("1.눈/비 2.맑음 3.흐림 "))
if num == 1: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall > 0) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall > 0);' 
elif num == 2: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud < 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud < 5);' 
else: 
    where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);' 

dataset_sql = 'SELECT grid, CCTV_num, police_value, resident, floating, Adult, Apartment, Commercial, Detached, School, Traffic FROM Dataset ' + where
grid_sql = 'SELECT grid, CCTV_num, police_value, resident, floating, Adult, Apartment, Commercial, Detached, School, Traffic FROM Grid WHERE time = \''+ time + '\''
curs.execute(dataset_sql)
dataset_result = curs.fetchall()
curs.execute(grid_sql)
grid_result = curs.fetchall()

1.Spring 2.Summer 3.Fall 4.Winter 3
1.Morning 2.Afternoon 3.Evening 4.Midnight 3
1.눈/비 2.맑음 3.흐림 3


In [3]:
# Transform to Pandas Dataframe
import pandas as pd
dataset_result = pd.DataFrame(dataset_result)
grid_result = pd.DataFrame(grid_result)
result = pd.concat([dataset_result, grid_result])
result['crime_num'] = result.groupby(['grid']).grid.transform('count') -1
result = result.drop_duplicates()
result = result.sort_values('grid')
result

,grid,CCTV_num,police_value,resident,floating,Adult,Apartment,Commercial,Detached,School,Traffic,crime_num
76,0,0,0,20,930,0,0,0,0,0,0,1
1,1,0,0,30,670,0,21,0,0,0,0,0
33,2,0,0,20,810,0,0,0,0,0,0,2
134,3,0,0,20,660,0,0,0,0,0,0,1
335,4,0,0,40,760,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
442,1102,8,0,310,6430,0,17,0,26,0,0,2
1103,1103,0,0,330,6250,0,0,0,0,0,0,0
423,1104,0,0,450,6400,0,0,0,0,0,0,2
1105,1105,0,0,240,6770,0,0,0,0,0,0,0


In [19]:
##############################################################################
################################ SVM ################################
##############################################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split

season_matrix = ["Spring", "Summer", "Fall", "Winter"]
time_matrix = ["Morning", "Afternoon", "Evening", "Midnight"]

accuracy = []

for s in season_matrix:
    season = s
    
    for t in time_matrix:
        time = t
    
        # Weather    
        for num in range(1, 4):
            if num == 1: 
                where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall > 0) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall > 0);' 
            elif num == 2: 
                where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud < 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud < 5);' 
            else: 
                where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);' 
            
            
            sql = 'SELECT grid, CCTV_num, police_value, resident, floating, Adult, Apartment, Commercial, Detached, School, Traffic FROM Dataset ' + where
            curs.execute(sql)
            result = curs.fetchall()
            
            # count 열 추가
            result = pd.DataFrame(result)
            result['crime_num'] = result.groupby(['grid']).grid.transform('count')

            # 학습해야하는 변수
            data = result[["CCTV_num", "police_value", "resident", "floating", "Adult", "Apartment", 
                           "Commercial", "Detached", "School", "Traffic"]].to_numpy()
            # 맞춰야하는 변수: crime_num
            target = result.crime_num.to_numpy()
            
            # 데이터셋 나누기
            X_train, X_test, y_train, y_test = train_test_split(data, target, stratify=result.crime_num, random_state=42)
            
            from sklearn.preprocessing import StandardScaler
 
            sc = StandardScaler()
            sc.fit(X_train)
 
            X_train_std = sc.transform(X_train)
            X_test_std = sc.transform(X_test)

            from sklearn.svm import SVC
 
            svm_model = SVC(kernel='rbf', C=8, gamma=0.1)
 
            svm_model.fit(X_train_std, y_train) # SVM 분류 모델 훈련

            y_pred = svm_model.predict(X_test_std) # 테스트

            #print("예측된 라벨:", y_pred)
            #print("ground-truth 라벨:", y_test)

            print(season + " " + time + " " + str(num) + " test accuracy: {:.2f}".format(np.mean(y_pred == y_test))) # 예측 정확도
            accuracy.append(np.mean(y_pred == y_test))
            
print("Average test accurage : " + str(np.mean(accuracy)))

Spring Morning 1 test accuracy: 0.84
Spring Morning 2 test accuracy: 0.47
Spring Morning 3 test accuracy: 0.51
Spring Afternoon 1 test accuracy: 0.95
Spring Afternoon 2 test accuracy: 0.46
Spring Afternoon 3 test accuracy: 0.42
Spring Evening 1 test accuracy: 0.92
Spring Evening 2 test accuracy: 0.41
Spring Evening 3 test accuracy: 0.57
Spring Midnight 1 test accuracy: 0.81
Spring Midnight 2 test accuracy: 0.45
Spring Midnight 3 test accuracy: 0.51
Summer Morning 1 test accuracy: 0.89
Summer Morning 2 test accuracy: 0.68
Summer Morning 3 test accuracy: 0.47
Summer Afternoon 1 test accuracy: 0.85
Summer Afternoon 2 test accuracy: 0.55
Summer Afternoon 3 test accuracy: 0.42
Summer Evening 1 test accuracy: 0.82
Summer Evening 2 test accuracy: 0.41
Summer Evening 3 test accuracy: 0.42
Summer Midnight 1 test accuracy: 0.86
Summer Midnight 2 test accuracy: 0.51
Summer Midnight 3 test accuracy: 0.55
Fall Morning 1 test accuracy: 0.91
Fall Morning 2 test accuracy: 0.53
Fall Morning 3 test accu

In [14]:
##############################################################################
################################ SVM ################################
##############################################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split

season = "Spring"
time = "Evening"
where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);' 


sql = 'SELECT grid, CCTV_num, police_value, resident, floating, Adult, Apartment, Commercial, Detached, School, Traffic FROM Dataset ' + where
curs.execute(sql)
result = curs.fetchall()

# count 열 추가
result = pd.DataFrame(result)
result['crime_num'] = result.groupby(['grid']).grid.transform('count')

# 학습해야하는 변수
data = result[["CCTV_num", "police_value", "resident", "floating", "Adult", "Apartment", 
               "Commercial", "Detached", "School", "Traffic"]].to_numpy()
# 맞춰야하는 변수: crime_num
target = result.crime_num.to_numpy()

# 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(data, target, stratify=result.crime_num, random_state=42)

print(X_train)

print("특성1 범위: ", "[", min(X_train[:, 0]), ",", max(X_train[:, 0]), "]")
print("특성2 범위: ", "[", min(X_train[:, 1]), ",", max(X_train[:, 1]), "]")
print("특성3 범위: ", "[", min(X_train[:, 2]), ",", max(X_train[:, 2]), "]")
print("특성4 범위: ", "[", min(X_train[:, 3]), ",", max(X_train[:, 3]), "]")
print("특성5 범위: ", "[", min(X_train[:, 4]), ",", max(X_train[:, 0]), "]")
print("특성6 범위: ", "[", min(X_train[:, 5]), ",", max(X_train[:, 1]), "]")
print("특성7 범위: ", "[", min(X_train[:, 6]), ",", max(X_train[:, 2]), "]")
print("특성8 범위: ", "[", min(X_train[:, 7]), ",", max(X_train[:, 3]), "]")
print("특성9 범위: ", "[", min(X_train[:, 8]), ",", max(X_train[:, 0]), "]")
print("특성10 범위: ", "[", min(X_train[:,9]), ",", max(X_train[:, 1]), "]")

from sklearn.preprocessing import StandardScaler
 
sc = StandardScaler()
sc.fit(X_train)
 
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

print("표준화된 특성1 범위: ", "[", min(X_train_std[:, 0]), ",", max(X_train_std[:, 0]), "]")
print("표준화된 특성2 범위: ", "[", min(X_train_std[:, 1]), ",", max(X_train_std[:, 1]), "]")
print("표준화된 특성3 범위: ", "[", min(X_train_std[:, 2]), ",", max(X_train_std[:, 2]), "]")
print("표준화된 특성4 범위: ", "[", min(X_train_std[:, 3]), ",", max(X_train_std[:, 3]), "]")
print("표준화된 특성5 범위: ", "[", min(X_train_std[:, 0]), ",", max(X_train_std[:, 4]), "]")
print("표준화된 특성6 범위: ", "[", min(X_train_std[:, 1]), ",", max(X_train_std[:, 5]), "]")
print("표준화된 특성7 범위: ", "[", min(X_train_std[:, 2]), ",", max(X_train_std[:, 6]), "]")
print("표준화된 특성8 범위: ", "[", min(X_train_std[:, 3]), ",", max(X_train_std[:, 7]), "]")
print("표준화된 특성9 범위: ", "[", min(X_train_std[:, 2]), ",", max(X_train_std[:, 8]), "]")
print("표준화된 특성10 범위: ", "[", min(X_train_std[:, 3]), ",", max(X_train_std[:, 9]), "]")

from sklearn.svm import SVC
 
svm_model = SVC(kernel='rbf', C=8, gamma=0.1)
 
svm_model.fit(X_train_std, y_train) # SVM 분류 모델 훈련

y_pred = svm_model.predict(X_test_std) # 테스트

print("예측된 라벨:", y_pred)
print("ground-truth 라벨:", y_test)

print("prediction accuracy: {:.2f}".format(np.mean(y_pred == y_test))) # 예측 정확도

[[  1   1  50 ...   4   0   0]
 [  0   0 100 ...   3   0   0]
 [  0   0 230 ...   0   0   0]
 ...
 [  3   1  30 ...   1   0   0]
 [  0   0  90 ...   0   0   1]
 [  0   0  60 ...   4   0   0]]
특성1 범위:  [ 0 , 15 ]
특성2 범위:  [ 0 , 5 ]
특성3 범위:  [ 0 , 480 ]
특성4 범위:  [ 180 , 7150 ]
특성5 범위:  [ 0 , 15 ]
특성6 범위:  [ 0 , 5 ]
특성7 범위:  [ 0 , 480 ]
특성8 범위:  [ 0 , 7150 ]
특성9 범위:  [ 0 , 15 ]
특성10 범위:  [ 0 , 5 ]
표준화된 특성1 범위:  [ -0.5538630098610293 , 4.801021066520486 ]
표준화된 특성2 범위:  [ -0.365363876985555 , 7.64906955366533 ]
표준화된 특성3 범위:  [ -1.2379323297509117 , 2.615774035825603 ]
표준화된 특성4 범위:  [ -1.215556026940067 , 1.5093730127000655 ]
표준화된 특성5 범위:  [ -0.5538630098610293 , 11.675148431993954 ]
표준화된 특성6 범위:  [ -0.365363876985555 , 18.334458058349306 ]
표준화된 특성7 범위:  [ -1.2379323297509117 , 14.03962700228582 ]
표준화된 특성8 범위:  [ -1.215556026940067 , 8.340798344595019 ]
표준화된 특성9 범위:  [ -1.2379323297509117 , 10.530298585800178 ]
표준화된 특성10 범위:  [ -1.215556026940067 , 10.56559831238905 ]
예측된 라벨: [1 1 2 1 3 2 1 